In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.cloud import bigquery

In [ ]:
client=bigquery.Client()

In [ ]:
dataset=client.dataset('hacker_news',project="bigquery-public-data")
dataset=client.get_dataset(dataset)

In [ ]:
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

In [ ]:
table_ref=dataset.table("full")

In [ ]:
table=client.get_table(table_ref)
table.schema

In [ ]:
client.list_rows(table,max_results=5).to_dataframe()

In [ ]:
dataset=client.dataset("openaq",project="bigquery-public-data")
data=client.get_dataset(dataset)
tables=list(client.list_tables(data))
for table in tables:
    print(table.table_id)
                       

In [ ]:
table_ref=dataset.table("global_air_quality")
table=client.get_table(table_ref)
client.list_rows(table,max_results=5).to_dataframe()

In [ ]:
query="""SELECT city
         From `bigquery-public-data.openaq.global_air_quality`
         WHERE country='US'"""

In [ ]:
client.query(query).to_dataframe()

In [ ]:
query="""SELECT country,COUNT(1) AS numposts
         From `bigquery-public-data.openaq.global_air_quality`
         GROUP BY country
         HAVING COUNT(country)>1000"""

In [ ]:
client.query(query).to_dataframe()

In [ ]:
client.query(query).to_dataframe().plot()

In [ ]:
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")


dataset = client.get_dataset(dataset_ref)


table_ref = dataset_ref.table("international_education")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")


dataset = client.get_dataset(dataset_ref)


table_ref = dataset_ref.table("international_education")


table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
code_count_query = """SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
                   FROM `bigquery-public-data.world_bank_intl_education.international_education`
                   WHERE year = 2016
                   GROUP BY indicator_name, indicator_code
                   HAVING COUNT(1) >= 175
                   ORDER BY COUNT(1) DESC """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

code_count_results = code_count_query_job.to_dataframe()

print(code_count_results.head())


In [ ]:
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [ ]:
tables=client.list_tables(dataset)
for table in tables:
    print(table.table_id)
table_ref=dataset.table('taxi_trips')
table=client.get_table(table_ref)
client.list_rows(table,max_results=5).to_dataframe()

In [ ]:
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT trip_miles,trip_seconds, EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01'AND
                         trip_start_timestamp< '2017-07-01' AND
                         trip_seconds > 0 AND 
                         trip_miles > 0
                         
               )
               SELECT hour_of_day,COUNT(1) AS num_trips,3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query,job_config=safe_config) 
speeds_result = speeds_query_job.to_dataframe() 
print(speeds_result)

In [ ]:
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
list_of_tables =list(client.list_tables(dataset))
list_of_tables=[table.table_id for table in list_of_tables]
print(list_of_tables)

In [ ]:
answers_table_ref = dataset_ref.table("posts_answers")
answers_table = client.get_table(answers_table_ref)
client.list_rows(answers_table, max_results=5).to_dataframe()

In [ ]:
questions_table_ref = dataset_ref.table("posts_questions")

questions_table = client.get_table(questions_table_ref)

client.list_rows(questions_table, max_results=5).to_dataframe()

In [ ]:
bigquery_experts_query = """SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query,job_config=safe_config)
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() 

print(bigquery_experts_results.head())
